<a href="https://colab.research.google.com/github/mg4603/HPC-survey-paper/blob/master/model_based_on_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, classification_report

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Dropout, ZeroPadding2D
from keras import Model
from keras.layers import Input
from keras.regularizers import l2

from keras.utils import np_utils
from keras import backend as K
import cv2

Using TensorFlow backend.


In [2]:

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'forse', 'ship', 'truck']



170500096/170498071 [==============================] - 2s 0us/step


In [0]:
num_classes = 10
input_shape = [28, 28]

X_val = X_train[-1000:,:]
y_val = y_train[-1000:]

X_train = X_train[:-1000, :]
y_train = y_train[:-1000]


In [0]:

X_train_upscaled = np.zeros((49000, 3, 227, 227))

for i, img in enumerate(X_train):
    im = img.reshape((1, 2, 0))
    large_img = cv2.resize(im, dsize=(227, 227), interpolation=cv2.INTER_CUBIC)
    X_train_upscaled[i] = large_img.reshape((2, 0, 1))

In [0]:
X_val_upscaled = np.zeros((1000, 3, 227, 227))

for i, img in enumerate(X_val):
    im = img.reshape((1, 2, 0))
    large_img = cv2.resize(im, dsize=(227, 227), interpolation=cv2.INTER_CUBIC)
    X_val_upscaled[i] = large_img.reshape((2, 0, 1))

In [0]:
X_test_upscaled = np.zeros((10000, 3, 227, 227))


for i, img in enumerate(X_test):
    im = img.reshape((1, 2, 0))
    large_img = cv2.resize(im, dsize=(227, 227), interpolation=cv2.INTER_CUBIC)
    X_test_upscaled[i] = large_img.reshape((2, 0, 1))


In [0]:

X_train_upscaled = X_train
X_val_upscaled = X_val
X_test_upscaled = X_test


X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image

print('X_train.shape =',X_train.shape)
print('X_val.shape   =',X_val.shape)
print('X_test.shape  =',X_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train.shape =',y_train.shape)
print('y_val.shape   =',y_val.shape)
print('y_test.shape  =',y_test.shape)

data = (X_train, y_train, X_val, y_val, X_test, y_test)


In [0]:
def try_model(model_name, data, epochs = 2, batch_size = 256 ):
    
    myModel = model_name()
    myModel.summary()
    myModel.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

    X_train, y_train, X_val, y_val, X_test, y_test = data
    
    myModel.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                validation_data=(X_val, y_val), verbose=2)
    
    scores = myModel.evaluate(X_train, y_train, verbose=2)
    print('Model Train',myModel.metrics_names[1],': %.2f%%' %(scores[1] * 100))

    scores = myModel.evaluate(X_val, y_val, verbose=2)
    print('Model Val',myModel.metrics_names[1],': %.2f%%' %(scores[1] * 100))
    
    y_pred = myModel.predict(X_test)
    print('Accuracy = %.2f%%' % (accuracy_score(y_test.argmax(1), y_pred.argmax(1))*100))
    
    return myModel, y_pred

In [0]:
def alexnet():
    return Sequential([
        # 1
        Conv2D(96, (11, 11), strides=4, input_shape=(227,227,3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        BatchNormalization(),
        
        # 2
        ZeroPadding2D((2, 2)),
        Conv2D(256, (5, 5), activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        BatchNormalization(),
        
        # 3
        ZeroPadding2D((1, 1)),
        Conv2D(384, (3, 3), activation='relu'),
        
        # 4
        ZeroPadding2D((1, 1)),
        Conv2D(384, (3, 3), activation='relu'),
        
        # 5
        ZeroPadding2D((1, 1)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        # 6
        Flatten(),
        Dense(4096, activation = 'relu'),
        Dropout(0.5),
        
        # 7
        Dense(4096, activation = 'relu'),
        Dropout(0.5),
        
        # 8
        Dense(num_classes, activation = 'softmax')
    ])


In [0]:
myModel, y_pred = try_model(alexnet, data)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
zero_padding2d_29 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 13, 13, 256)     

ValueError: ignored